# **任务三 多层感知机单分类任务 MLP binary classification task**

对于回归和分类任务在代码上的主要体现，仅在输出维度和损失函数上存在差异。

## 1. 模型定义

依然使用最简单的多层感知机进行实验。

由于分类任务，需要将连续的输出数值映射到一个不连续的离散空间，所以这里用到 `nn.Sigmoid()` 激活函数，它会将一个数值映射到0到1的值域空间中，只需要在最后设置一个阈值作为判断就可以将概率映射为类别。

In [562]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.Linear_layer0 = nn.Linear(input_dim, hidden_dim)
        self.Linear_layer1 = nn.Linear(hidden_dim, hidden_dim)
        self.Linear_layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.Linear_layer3 = nn.Linear(hidden_dim, hidden_dim)
        self.Linear_layer4 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.Linear_layer0(x)
        x = self.relu(x)
        x = self.Linear_layer1(x)
        x = self.relu(x)
        x = self.Linear_layer2(x)
        x = self.relu(x)
        x = self.Linear_layer3(x)
        x = self.relu(x)
        x = self.Linear_layer4(x)
        x = self.sigmoid(x)
        return x

## 2. 数据生成

单分类任务的模型输出需要映射成一个连续概率值，但实际目标值标签是一个离散标签，所以使用0.0视为否，1.0视为是这种极端赋值的方式来标记数据。

任务目标：基于二维坐标特征 (x1, x2)，判断样本是否落在「以原点为圆心、半径 2 的圆内」（正例：y=1；负例：y=0）。
核心优势：边界是严格的几何规则（x1² + x2² ≤ 4），无随机得分波动，样本生成高效且标签绝对准确，MLP 可快速拟合非线性边界（圆是二次曲线，MLP 单隐藏层即可拟合）。

In [563]:
import numpy as np

def get_data(data_size):
    n_samples = data_size if data_size % 2 == 0 else data_size - 1
    n_pos = n_neg = n_samples // 2

    # --------------------- 1. 生成正例（圆内：x1² + x2² ≤ 4） ---------------------
    # 极坐标生成圆内均匀样本（避免随机采样的密度不均）
    pos_r = np.random.uniform(0, 2, n_pos)  # 半径0~2
    pos_theta = np.random.uniform(0, 2*np.pi, n_pos)  # 角度0~2π
    pos_x1 = pos_r * np.cos(pos_theta)
    pos_x2 = pos_r * np.sin(pos_theta)
    pos_x = np.vstack([pos_x1, pos_x2]).T  # (n_pos, 2)

    # --------------------- 2. 生成负例（圆环区：4 < x1² + x2² ≤ 25） ---------------------
    neg_r = np.random.uniform(2, 5, n_neg)  # 半径2~5
    neg_theta = np.random.uniform(0, 2*np.pi, n_neg)
    neg_x1 = neg_r * np.cos(neg_theta)
    neg_x2 = neg_r * np.sin(neg_theta)
    neg_x = np.vstack([neg_x1, neg_x2]).T  # (n_neg, 2)

    # --------------------- 3. 合并并打乱 ---------------------
    all_x = np.vstack([pos_x, neg_x])
    all_y = np.vstack([np.ones((n_pos, 1)), np.zeros((n_neg, 1))])

    # 随机打乱样本顺序
    shuffle_idx = np.random.permutation(n_samples)
    all_x = all_x[shuffle_idx]
    all_y = all_y[shuffle_idx]

    # 转换为PyTorch张量（float32，适配MLP）
    x = torch.tensor(all_x, dtype=torch.float32)
    y = torch.tensor(all_y, dtype=torch.float32)

    return x, y


# 划分批次
def split_batch(data, batch_size):
    # 核心操作：沿第一个维度（dim=0）分割，保留后续所有维度
    split_tensors = torch.split(data, batch_size, dim=0)
    # 转为列表返回（torch.split返回tuple，列表更易操作）
    return list(split_tensors)

# 训练数据
batch_size = 128
train_x, train_y = get_data(1024)
train_x_batch = split_batch(train_x, batch_size)
train_y_batch = split_batch(train_y, batch_size)
# 验证数据
val_x, val_y = get_data(128)
# 测试数据
test_x, test_y = get_data(6)
print('输入数据形状:', train_x.shape)
print('输入批次数量:', len(train_x_batch), '\t批次形状:', train_x_batch[0].shape)
print('标签数据形状:', train_y.shape)
print('输入批次数量:', len(train_y_batch), '\t批次形状:', train_y_batch[0].shape)


输入数据形状: torch.Size([1024, 2])
输入批次数量: 8 	批次形状: torch.Size([128, 2])
标签数据形状: torch.Size([1024, 1])
输入批次数量: 8 	批次形状: torch.Size([128, 1])


## 3. 模型训练

### 3.1 实例化模型、损失函数、优化器

本次任务使用二元交叉熵损失（BCELoss） `nn.BCELoss()`，这是二分类任务的关键。

In [564]:
model = Model(2, 1, 32)
criterion = nn.BCELoss()  # 匹配带sigmoid的模型输出
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

### 3.2 **迭代训练**

In [565]:
epochs = 1000
for epoch in range(epochs):
    loss = None
    for i in range(len(train_x_batch)):
        x = train_x_batch[i]
        y = train_y_batch[i]
        model.train()
        # 前向传播，得到预测值
        output = model(x)
        # 计算损失
        loss = criterion(output, y)
        # 梯度清零，因为在每次反向传播前都要清除之前累积的梯度
        optimizer.zero_grad()
        # 反向传播，计算梯度
        loss.backward()
        # 更新权重和偏置
        optimizer.step()

    model.eval()
    output = model(val_x)
    val_loss = criterion(output, val_y).item()

    # 更改验证逻辑为适合分类任务的准确率和召回率
    ace = torch.sum((output > 0.5) == val_y).item() / val_y.shape[0]
    recall = (torch.sum((output > 0.5) & (val_y == 1))).item() / max(torch.sum(val_y).item(), 1)
    if (epoch + 1) % 200 == 0:
        print(f'[epoch {epoch+1}]loss:', loss.item())
        print(f'\t val loss:', val_loss)
        print(f'\t val ace: {ace * 100:.2f}%\trecall: {recall * 100:.2f}%')


[epoch 200]loss: 0.6772708296775818
	 val loss: 0.6758170127868652
	 val ace: 50.00%	recall: 0.00%
[epoch 400]loss: 0.6415574550628662
	 val loss: 0.638750433921814
	 val ace: 57.03%	recall: 14.06%
[epoch 600]loss: 0.5585832595825195
	 val loss: 0.5540313720703125
	 val ace: 71.88%	recall: 43.75%
[epoch 800]loss: 0.4396541714668274
	 val loss: 0.43794944882392883
	 val ace: 82.81%	recall: 65.62%
[epoch 1000]loss: 0.33800557255744934
	 val loss: 0.33733099699020386
	 val ace: 94.53%	recall: 89.06%


### 3.3 **测试模型**

使用 `model.eval()` 将模型改为测试模式，避免自动的梯度计算增加额外的计算量。

In [566]:
model.eval()
output = model(test_x)

for i in range(len(test_x)):
    print('输入数据:', test_x[i].tolist())
    print('目标结果:', test_y[i].item())
    print('预测结果:', (output[i] > 0.5).item())

输入数据: [-1.1521484851837158, 0.23579376935958862]
目标结果: 1.0
预测结果: True
输入数据: [1.6813095808029175, 0.9599031805992126]
目标结果: 1.0
预测结果: False
输入数据: [-0.3279746174812317, -3.5669851303100586]
目标结果: 0.0
预测结果: False
输入数据: [-1.0948398113250732, -0.6206879019737244]
目标结果: 1.0
预测结果: True
输入数据: [-0.22980260848999023, 2.611698627471924]
目标结果: 0.0
预测结果: False
输入数据: [-0.4656290113925934, -3.873504400253296]
目标结果: 0.0
预测结果: False


## 4. **总结**

这是第三个关于 `torch` 框架的任务，第一次引入了分类任务的概念，区别于以往的输出一个连续数值的任务，本次任务的目标是划分两种离散的状态。